In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

In [ ]:
!pip install -U sentence-transformers

In [ ]:
!pip install bert-tensorflow

In [ ]:
!pip install tensorflow-gpu==1.15.0
!pip install bert-tensorflow
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [ ]:
!wget "https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml"

--2021-01-28 19:00:32--  https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml
Resolving ir.nist.gov (ir.nist.gov)... 129.6.24.92, 2610:20:6005:24::92
Connecting to ir.nist.gov (ir.nist.gov)|129.6.24.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18707 (18K) [application/xml]
Saving to: ‘topics-rnd5.xml’

topics-rnd5.xml     100%[===================>]  18.27K  --.-KB/s    in 0.01s   

2021-01-28 19:00:32 (1.82 MB/s) - ‘topics-rnd5.xml’ saved [18707/18707]



In [ ]:
!wget "https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt"

--2021-01-28 20:08:29--  https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt
Resolving ir.nist.gov (ir.nist.gov)... 129.6.24.92, 2610:20:6005:24::92
Connecting to ir.nist.gov (ir.nist.gov)|129.6.24.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1142244 (1.1M) [text/plain]
Saving to: ‘qrels-covid_d5_j0.5-5.txt’

qrels-covid_d5_j0.5 100%[===================>]   1.09M  1.72MB/s    in 0.6s    

2021-01-28 20:08:30 (1.72 MB/s) - ‘qrels-covid_d5_j0.5-5.txt’ saved [1142244/1142244]



In [ ]:
FILE_EVALUATE = "qrels-covid_d5_j0.5-5.txt"

In [ ]:
import os

def searchExistingFile(file_name):
    return os.path.exists(file_name)

def getCorpusList(docs:dict):
    docIds = getDocIds(docs) # Sorted
    corpus = list()
    for docId in docIds:
        textList = list()
        title = docs[docId]['TITLE']
        abstract = docs[docId]['ABSTRACT']
        for elem in title:
            textList.append(elem)
        for elem in abstract:
            textList.append(elem)
        corpus.append(' '.join(textList))
    return corpus

def getDocIds(docs:dict):
    return sorted(docs.keys())

In [ ]:
import os
import sys
import pickle
import csv

# REFERENCE: https://stackoverflow.com/questions/52937859/read-csv-file-line-by-line-python
def tokenization(file_name='/content/drive/My Drive/metadata.csv'):
    docs = dict()
    with open (file_name,'r') as csv_file:
        reader =csv.reader(csv_file)
        next(reader) # skip first row
        for row in reader:
            cord_uid = row[0]
            title = row[3]
            abstract = row[8]
            if cord_uid in docs:
                if len(title) > len(docs[cord_uid]['TITLE']):
                    docs[cord_uid]['TITLE'] = title
                if len(abstract) > len(docs[cord_uid]['ABSTRACT']):
                    docs[cord_uid]['ABSTRACT'] = abstract
            else:
                docs[cord_uid] = {'TITLE': title,
                              'ABSTRACT': abstract}
    for docId in docs:
        title = docs[docId]['TITLE']
        abstract = docs[docId]['ABSTRACT']
        docs[docId]['TITLE'] = title.split()
        docs[docId]['ABSTRACT'] = abstract.split()
    return docs

def saveDocs(docs,fileName = '/content/drive/My Drive/TREC-COVID_DOCS.pickle'):
    with open(fileName, 'wb') as f:
        pickle.dump(docs, f, protocol=pickle.HIGHEST_PROTOCOL)

def loadDocs(fileName = '/content/drive/My Drive/TREC-COVID_DOCS.pickle'):
    with open(fileName, 'rb') as f:
        return pickle.load(f)

In [ ]:
import math
import string
import xml.etree.ElementTree as ET
import json
import pickle

def getQueries():
    tree = ET.parse('topics-rnd5.xml')
    root = tree.getroot()
    queryDict = dict()
    for child in root.findall('topic'):
        id = child.get('number')
        queryDict[id] = {}
        queryDict[id]['query'] = child.find('query').text
        queryDict[id]['question'] = child.find('question').text
        queryDict[id]['narrative'] = child.find('narrative').text
    return queryDict

def getQuerySet(subset:str):
    queries = getQueries()
    querySet = dict()
    if subset.lower() == "training":
        for key in queries:
            if int(key) % 2 == 1:
                querySet[key] = queries[key]['query'] + " " + queries[key]['question'] + " " + queries[key]['narrative']
    elif subset.lower() == "test":
        for key in queries:
            if int(key) % 2 == 0:
                querySet[key] = queries[key]['query'] + " " + queries[key]['question'] + " " + queries[key]['narrative']
    else:
        for key in queries:
            querySet[key] = queries[key]['query'] + " " + queries[key]['question'] + " " + queries[key]['narrative']
    return querySet

In [ ]:
dict_evaluate = dict()
def fill_evaluate():
  with open(FILE_EVALUATE, 'r') as f:
    lines = f.readlines()
    for line in lines:
      list_temp = line.split()
      query_id = list_temp[0]
      if query_id not in dict_evaluate:
        dict_evaluate[query_id] = list()
        dict_evaluate[query_id].append(list_temp[2])
      else:
        dict_evaluate[query_id].append(list_temp[2])

fill_evaluate()    
  

In [ ]:
def getResultsFile(topKResults:dict):
    # FORMAT: query-id Q0 document-id rank score STANDARD
    with open('/content/drive/My Drive/RESULTS', 'w') as f:
        for queryId in topKResults:
            for i, resTuple in enumerate(topKResults[queryId]):
                docId = resTuple[0]
                if docId in dict_evaluate[queryId]:
                  cosineSimilarity = resTuple[1]
                  print("%s\tQ0\t%s\t%d\t%f\tSTANDARD" % (queryId,docId,rank,cosineSimilarity), file=f)
                rank = i + 1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
isDocsFound = searchExistingFile('/content/drive/My Drive/TREC-COVID_DOCS.pickle')
print("Is 'TREC-COVID_DOCS.pickle' found?: ",isDocsFound)

if not isDocsFound:
    docs = tokenization()
    print("Tokenization has been successfully done...")
    saveDocs(docs)

Is 'TREC-COVID_DOCS.pickle' found?:  False
Tokenization has been successfully done...


In [ ]:
docs = loadDocs()
docIds = getDocIds(docs)
corpus = getCorpusList(docs)
del docs

In [ ]:
print("Corpus loaded.")
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
document_embeddings = sbert_model.encode(corpus)
print("Document embeddings calculated.")

Corpus loaded.


100%|██████████| 405M/405M [00:14<00:00, 27.8MB/s]


Document embeddings calculated.


In [ ]:
querySet = getQuerySet("all")

In [ ]:
results = dict()
for queryId, queryStr in querySet.items():
    query_embedding = sbert_model.encode(queryStr)
    cosineSimilarityVector = cosine_similarity(query_embedding,document_embeddings)[0]
    queryResult = list()
    for docId, cosineSimilarity in zip(docIds, cosineSimilarityVector):
        queryResult.append([docId,cosineSimilarity])
    results[queryId] = queryResult

In [ ]:
# Rank documents for each query by cosineSimilarity. (For analysis purposes)
for queryId in results:
    resultsList = results[queryId]
    resultsList.sort(key=lambda x: x[1], reverse=True)
    results[queryId] = resultsList

In [ ]:
getResultsFile(results)